In [ ]:
!curl -s -OL https://raw.githubusercontent.com/StephenZou-bot/sd-webui-notebook/main/cui/base.py
!curl -s -OL https://raw.githubusercontent.com/StephenZou-bot/sd-webui-notebook/main/cui/get_ip.py
!curl -s -OL https://raw.githubusercontent.com/StephenZou-bot/sd-webui-notebook/main/cui/pastebin.py
%run base.py

### Download from Pastebin URL

In [ ]:
pastebin_url = "" # example link
hf_token = "" # if use private repo
civitai_api_key = ""
#=====================================================
%run pastebin.py --pastebin_url {pastebin_url}  --hf_token {hf_token} --civitai_api_key {civitai_api_key}


## Link Dataset

In [ ]:
import os

def link_kaggle_files(input_dir, output_dir):
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(input_dir):

        relative_path = os.path.relpath(root, input_dir)

        target_root = os.path.join(output_dir, relative_path)
        if not os.path.exists(target_root):
            os.makedirs(target_root)

        for file in files:
            source_file = os.path.join(root, file)
            target_file = os.path.join(target_root, file)
            if not os.path.exists(target_file):
                os.symlink(source_file, target_file)
            else:
                print(f"Skipping existing file: {target_file}")

datasets = {
    "/kaggle/input/checkpoints": f"{ui}/ComfyUI/models/checkpoints/",
    "/kaggle/input/checkpoint": f"{ui}/ComfyUI/models/controlnet/",
    "/kaggle/input/load-vae": f"{ui}/ComfyUI/models/vae/"
}

for input_dir, output_dir in datasets.items():
    link_kaggle_files(input_dir, output_dir)
    print(f"Linked files from {input_dir} to {output_dir}")


## Run WebUI

In [ ]:
ngrok_token = "" # https://dashboard.ngrok.com/get-started/your-authtoken
zrok_token = "" # optional, good ngrok alternative

#================================================
%run /kaggle/working/get_ip.py

tunnel_port= 8188
tunnel = Tunnel(tunnel_port)

tunnel.add_tunnel(command="cl tunnel --url localhost:{port}", name="cl", pattern=re.compile(r"[\w-]+\.trycloudflare\.com"))
tunnel.add_tunnel(command="lt --port {port}", name="lt", pattern=re.compile(r"[\w-]+\.loca\.lt"), note="Password : " + Fore.GREEN + public_ipv4 + Style.RESET_ALL + " rerun cell if 404 error.")
if zrok_token:
    !zrok enable {zrok_token} &> /dev/null
    tunnel.add_tunnel(command="zrok share public http://localhost:{port}/ --headless", name="zrok", pattern=re.compile(r"[\w-]+\.share\.zrok\.io"))

with tunnel:
    #!python -m http.server 1101
    %cd {ui}/ComfyUI
    !echo -n {start_colab} >{ui}/ComfyUI/colabTimer.txt
    !venv/bin/python main.py --dont-print-server --preview-method auto --enable-cors-header --use-pytorch-cross-attention